In [1]:
import kagglehub
import numpy as np

# Download latest version
path = kagglehub.dataset_download("rmisra/news-headlines-dataset-for-sarcasm-detection")

print("Path to dataset files:", path)

Path to dataset files: /home/zanchet/.cache/kagglehub/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection/versions/2


In [2]:
import json

sentences = []
labels = []
urls = []
with open(path + "/Sarcasm_Headlines_Dataset.json", 'r') as f:
    for line in f:
        data = json.loads(line)
        sentences.append(data['headline'])
        labels.append(data['is_sarcastic'])
        urls.append(data['article_link'])

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-12-05 15:01:55.808533: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-05 15:01:55.837166: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733421715.873663   84493 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733421715.885433   84493 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 15:01:55.924952: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)

In [5]:
word_index = tokenizer.word_index
vocab_size = len(word_index)

In [6]:
sequences = tokenizer.texts_to_sequences(sentences)

In [7]:
pad_lenght = 40
padded = pad_sequences(sequences, padding='post', maxlen=pad_lenght)

In [8]:
len(padded[0])

40

In [9]:
padded.shape

(26709, 40)

In [10]:
validation_percentage = 15

ts = int(padded.shape[0] * (100 - validation_percentage) / 100)

X_train = padded[0:ts]
X_test = padded[ts:]

y_train = labels[0:ts]
y_test = labels[ts:]

In [11]:
X_train.shape

(22702, 40)

In [12]:
X_test.shape

(4007, 40)

In [13]:
import tensorflow as tf


In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 600, input_length = X_train.shape[1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_11                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_12                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
num_epochs = 30

In [29]:
history = model.fit(np.array(X_train), np.array(y_train), epochs = num_epochs, validation_data=(np.array(X_test), np.array(y_test)), verbose=2)

Epoch 1/30
710/710 - 31s - 43ms/step - accuracy: 0.8183 - loss: 0.3938 - val_accuracy: 0.8755 - val_loss: 0.2978
Epoch 2/30
710/710 - 25s - 35ms/step - accuracy: 0.9495 - loss: 0.1405 - val_accuracy: 0.8665 - val_loss: 0.3514
Epoch 3/30
710/710 - 24s - 34ms/step - accuracy: 0.9832 - loss: 0.0507 - val_accuracy: 0.8630 - val_loss: 0.5365
Epoch 4/30
710/710 - 23s - 33ms/step - accuracy: 0.9927 - loss: 0.0222 - val_accuracy: 0.8543 - val_loss: 0.7285
Epoch 5/30
710/710 - 24s - 33ms/step - accuracy: 0.9960 - loss: 0.0127 - val_accuracy: 0.8513 - val_loss: 0.8758
Epoch 6/30
710/710 - 24s - 33ms/step - accuracy: 0.9983 - loss: 0.0074 - val_accuracy: 0.8488 - val_loss: 0.9812
Epoch 7/30


KeyboardInterrupt: 

In [34]:
sentences = [
    "Hi, I am Lucas, a Computer Engineering student",
    "granny starting to feel spiders in the garden might be real",
    "the weather today is bright and sunny"
]

In [35]:
sequences = tokenizer.texts_to_sequences(sentences)

In [36]:
padded = pad_sequences(sequences, maxlen=pad_lenght, padding="post")

In [37]:
model.predict(padded)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 616ms/step


array([[1.1405050e-05],
       [4.1137498e-02],
       [8.0866251e-08]], dtype=float32)